In [8]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

torch.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the data

In [291]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


## Pre-processing

In [303]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


## Linear Regression

In [ ]:
lr = linear_model.LinearRegression()

X_train = train_input.view(train_input.size(0), -1)
X_test = test_input.view(test_input.size(0), -1)

# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
lr.fit(X=X_train, y=train_target)
preds = lr.predict(X_test)

preds = torch.FloatTensor(preds)

In [ ]:
preds = discrete_predictions(preds)
accuracy = compute_accuracy(test_target, preds)
print('Accuracy ' + str(accuracy*100) + '%')

## Neural Network

In [ ]:
# load data
train_input , train_target = bci.load(root = './data_bci')
test_input , test_target = bci.load(root = './data_bci', train = False)

Reduce the number of samples from 316 to 300 so that we can take bigger minibatches:

In [248]:
train_input2 = train_input[0:300]
train_target2 = train_target[0:300]

In [104]:
class Net(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, 2)

    def forward(self, x):
        #print("======")
        print(x.size())
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        print(x.size())
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        print(x.size())
        x = F.tanh(self.fc1(x.view(-1, 640)))
        print(x.size())
        #print("======")
        return x

In [ ]:
my_cool_net = Net()
train_model(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4, nb_epochs=50, learning_rate=1e-2, verbose=True)

#### Compute error rate

In [ ]:
nb_errors = compute_nb_errors(my_cool_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)

In [ ]:
nb_errors/test_input.size(0)

In [ ]:
train_errors = compute_nb_errors(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
train_errors / train_input.size(0)

### Grid search on model parameters

In [ ]:
step_sizes = [0.01, 0.005, 0.0025, 0.001, 0.0005]
nb_iters_ = [25, 50, 100, 250, 500]

test_errors = []
train_errors = []

for lr in step_sizes:
    test_errs = []
    train_errs = []
    for nb_iters in nb_iters_:
        # reset and train the network
        my_net = Net()
        train_model(my_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), mini_batch_size=4, nb_epochs=nb_iters, learning_rate=lr)
        
        # compute the number of errors
        n_train_errors = compute_nb_errors(my_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
        n_test_errors = compute_nb_errors(my_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
        
        train_error = n_train_errors / train_input.size(0)
        test_error = n_test_errors / test_input.size(0)
        
        test_errs.append(test_error)
        train_errs.append(train_error)
        
        print('step size: ' + str(lr) + ' nb epochs: ' + str(nb_iters) + ' train error: ' + str(train_error) + ' test error: ' + str(test_error))
    test_errors.append(test_errs)
    train_errors.append(train_errs)

## LSTM 

In [304]:
class Net2(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net2, self).__init__()
        self.inp = nn.Linear(50, 32)
        self.lstm = nn.LSTM(32, 32, 2, dropout=0.05)
        #self.lstm2 = nn.LSTM(64, 64, 2, dropout=0.05)
        self.out = nn.Linear(28*32, 2)

    def forward(self, x, hc=None):
        x = F.tanh(self.inp(x))
        x, hidden = self.lstm(x.squeeze(1), hc)
        x = F.tanh(x)
        #x, hidden2 = self.lstm2(x.squeeze(1), hidden)
        #x = F.tanh(x)
        x = F.tanh(self.out(x.view(-1, 28*32)))
        return x

In [305]:
torch.manual_seed(1)
model = Net2()
train_model(model,
            Variable(train_input.view(-1, 1, 28, 50)),
            Variable(train_target),
            mini_batch_size=4,
            nb_epochs=100,
            learning_rate=0.1,
            verbose=True)

0 55.869204103946686
1 55.43167996406555
2 55.19314759969711
3 55.059707939624786
4 54.94787919521332
5 54.82654511928558
6 54.69320738315582
7 54.43547594547272
8 54.04499363899231
9 53.71025663614273
10 53.101887077093124
11 52.36088967323303
12 51.46517211198807
13 50.23881909251213
14 48.79900071024895
15 47.382527619600296
16 46.05978602170944
17 44.92688646912575
18 43.7837099134922
19 42.69750751554966
20 41.3571610301733
21 39.86741977930069
22 38.91183651983738
23 38.50012291967869
24 37.5086385756731
25 36.48665054142475
26 35.347092255949974
27 35.29541324079037
28 33.63038110733032
29 33.86552679538727
30 32.97670891880989
31 32.42755489051342
32 31.461542561650276
33 30.93731053173542
34 29.90556702017784
35 28.646388962864876
36 28.038339361548424
37 26.947852313518524
38 26.298431307077408
39 24.39644142985344
40 23.045839563012123
41 22.45906500518322
42 21.94350579380989
43 21.287144765257835
44 20.972183883190155
45 20.335538625717163
46 19.979054808616638
47 18.90446

In [306]:
nb_errors = compute_nb_errors(model, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
1-nb_errors/test_input.size(0)

0.6799999999999999

In [307]:
train_errors = compute_nb_errors(model, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
1 - train_errors / train_input.size(0)

0.990506329113924

### Try window slicing

In [286]:
import dlc_bci as bci
train_input_big , train_target_big = bci.load(root = './data_bci', one_khz=True)
print(str(type(train_input_big)), train_input_big.size()) 
print(str(type(train_target_big)), train_target_big.size())
test_input_big , test_target_big = bci.load(root = './data_bci', train = False, one_khz=True)
print(str(type(test_input_big)), test_input_big.size()) 
print(str(type(test_target_big)), test_target_big.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [289]:
train_input_big.size()

torch.Size([316, 28, 500])

In [290]:
train_input_big.view(3160, 28, 50)


( 0  ,.,.) = 
   30.8000   33.1000   37.9000  ...    39.4000   41.3000   45.0000
   46.2000   42.9000   37.6000  ...    62.4000   59.3000   55.7000
   55.1000   59.7000   67.7000  ...    67.1000   68.4000   73.9000
              ...                ⋱                ...             
   84.4000   78.2000   67.3000  ...    35.8000   32.3000   23.9000
   19.5000   25.3000   37.0000  ...    76.4000   82.6000   88.7000
   87.9000   78.9000   67.9000  ...    58.3000   54.3000   45.9000

( 1  ,.,.) = 
   41.7000   47.8000   59.4000  ...    69.9000   71.3000   77.1000
   79.1000   72.4000   62.3000  ...    64.1000   62.0000   55.9000
    6.0000    9.9000   18.2000  ...    26.5000   28.3000   32.5000
              ...                ⋱                ...             
   45.0000   44.0000   38.4000  ...    28.0000   24.7000   20.8000
   21.1000   26.8000   34.2000  ...    33.6000   34.0000   36.7000
   39.7000   40.1000   36.9000  ...    27.7000   22.4000   13.7000

( 2  ,.,.) = 
    7.8000    8.9